In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd
from rasterio.sample import sample_gen
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,ConfusionMatrixDisplay,\
                            precision_score,recall_score,f1_score,roc_auc_score, roc_curve

# import matplotlib as plt
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier  #https://github.com/sentinel-hub/eo-learn-examples/blob/main/custom-script/machine-learning-evalscript.ipynb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)
from sklearn import svm
from sklearn.svm import LinearSVC

In [2]:
# Load the CSV data
# data = pd.read_csv('training_data_python_textures.csv')
training_data_path="output/sample_feature/palisades_sar_avg_indices.csv"
training_data=pd.read_csv(training_data_path)

# get from feature_extration.ipynb
# columns_names=["gam_vh_post","gam_vv_post","gam_vh_pre","gam_vv_pre"]
# columns_names=["gam_vh_post","gam_vv_post","gam_vh_pre","gam_vv_pre","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI"]
# columns_names= feature_column_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'] #,"ΔRVI",'Δvv_vh_ratio',"ΔRDFI","dTRAD"

columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI"]

# columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI"]
# columns_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']

# columns_names=["gamm_vh_pre","gamm_vv_pre","gamm_vh_post","gamm_vv_post","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']


In [3]:
# Separate features (SAR bands) and labels (e.g., class column)
X = training_data[columns_names]  # Features (SAR band values)
Y = training_data['class']  # Labels (add 'Class' column in your CSV if available)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


In [4]:
#To visualize data in X_train
pd.DataFrame(X_train)

,RBD_VV,RBD_VH,RBR_VV,RBR_VH,ΔRVI
1887,0.023507,-0.004842,0.172246,-0.184616,-2.984644
1316,0.002959,0.002316,0.026657,0.081230,0.434274
1934,0.062192,-0.047101,0.091722,-0.513127,-5.324075
1914,-0.016399,-0.003495,-0.130978,-0.125334,0.046158
1602,-0.010810,-0.005176,-0.084105,-0.133334,-0.378274
...,...,...,...,...,...
1638,0.043520,-0.002792,0.127984,-0.030979,-1.254833
1095,-0.011624,-0.006761,-0.088984,-0.211854,-0.988640
1130,-0.024352,-0.022959,-0.140083,-0.602166,-3.819186
1294,-0.022602,-0.005201,-0.274252,-0.315480,-0.343850


## To choose the models for hyper parameter tuning

In [5]:
models={
    "Random Forest":RandomForestClassifier(),
    # "Decision Tree":DecisionTreeClassifier(),
    # "Logistic Regression":LogisticRegression(), #NOT GOOD
    # "GradientBoost Classifier": GradientBoostingClassifier(), #NOT GOOD
    # "Light GBM":LGBMClassifier(),
    # "XGB Classifier":XGBClassifier(),
    # "LinearSVC":LinearSVC()

}

for i in range(len(list(models))):

    model=list(models.values())[i]
    model.fit(X_train, y_train)

    #Make a prediction
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    #Training set performance
    model_train_accuracy=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred,average='weighted')
    model_train_precision=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_roc_auc_score=roc_auc_score(y_train,y_train_pred)

    #Test set performance
    model_test_accuracy=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred,average='weighted')
    model_test_precision=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_roc_auc_score=roc_auc_score(y_test,y_test_pred)

    # print(model)
    print('Model performance for Training set')
    print(f"- Accuracy: {model_train_accuracy}")
    print(f"- F1 Score: {model_train_f1}")
    print(f"- Precision Score: {model_train_precision}")
    print(f"- Recall Score: {model_train_recall}")
    print(f"- Roc Auc Score: {model_train_roc_auc_score}")

    print("--------------")
    print('Model performance for Test set')
    print(f"- Accuracy: {model_test_accuracy}")
    print(f"- F1 Score: {model_test_f1}")
    print(f"- Precision Score: {model_test_precision}")
    print(f"- Recall Score: {model_test_recall}")
    print(f"- Roc Auc Score: {model_test_roc_auc_score}")
    print("----------------------------------------")


# randomforest and decisiontrees will be option for further hyperparameter tuning

Model performance for Training set
- Accuracy: 1.0
- F1 Score: 1.0
- Precision Score: 1.0
- Recall Score: 1.0
- Roc Auc Score: 1.0
--------------
Model performance for Test set
- Accuracy: 0.8359486447931527
- F1 Score: 0.8343828027986385
- Precision Score: 0.8817204301075269
- Recall Score: 0.75
- Roc Auc Score: 0.8307640750670241
----------------------------------------


In [ ]:
# #Hyperparameter training
# rf_params={"max_depth":[5,8,15,12,10],
#            "max_features":[5,7,1,2,3,4,9,12,20,30,40],
#            "min_samples_split":[2,8,15,20,30,40],
#            "n_estimators":[100,150,200,300]
#            }

rf_params = {
    'n_estimators': [50, 100,150, 200],  # Number of trees
    'max_depth': [10, 20, None],  # Depth of trees
    'min_samples_split': [2, 10, 40],  # Minimum samples to split
    'max_features': [2, 'sqrt', 'log2'],  # Features to consider at each split
    'class_weight': ['balanced_subsample', 'balanced'],  # Handle class imbalance
    'max_samples': [0.3, 0.5, None],  # Fraction of samples to train each tree on
}

#best param {'n_estimators': 50, 'max_depth': 20, 'learning_rate': 0.1}
# bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
xgboost_params={"max_depth":[10,20,30,40],
        #    "max_features":[5,7,8,9],
           "learning_rate":[0.001,0.01,0.5,0.1,1],
           "n_estimators":[32,50,100,200]
           }
#models list for hyperparameter tuning

randomcv_models=[
    ("RF",RandomForestClassifier(), rf_params),
    # ("XGBoost",XGBClassifier(), xgboost_params),

    # ("DT",DecisionTreeClassifier(), dt_params),
]


In [ ]:
model_param={}
# RandomizedSearchCV
for name, model, params in randomcv_models:
    random=GridSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=3,verbose=2,n_jobs=-1)
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_

for model_name in model_param:
    print(f"best param for {model_name}-----")
    print(model_param[model_name])

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END max_depth=5, max_features=9, min_samples_split=8, n_estimators=100; total time=   0.3s
[CV] END max_depth=5, max_features=9, min_samples_split=8, n_estimators=100; total time=   0.2s
[CV] END max_depth=5, max_features=9, min_samples_split=8, n_estimators=100; total time=   0.2s
[CV] END max_depth=8, max_features=9, min_samples_split=15, n_estimators=100; total time=   0.3s
[CV] END max_depth=5, max_features=2, min_samples_split=15, n_estimators=200; total time=   0.2s
[CV] END max_depth=5, max_features=2, min_samples_split=15, n_estimators=200; total time=   0.2s
[CV] END max_depth=8, max_features=9, min_samples_split=15, n_estimators=100; total time=   0.3s
[CV] END max_depth=5, max_features=2, min_samples_split=15, n_estimators=200; total time=   0.3s
[CV] END max_depth=12, max_features=1, min_samples_split=40, n_estimators=100; total time=   0.1s
[CV] END max_depth=12, max_features=1, min_samples_split=40, n_est

In [ ]:
# https://www.datacamp.com/tutorial/random-forests-classifier-python
# from scipy.stats import randint
# param_dist = {'n_estimators': randint(50,500),
#               'max_depth': randint(1,20)}

# # Create a random forest classifier
# rf = RandomForestClassifier()

# # Use random search to find the best hyperparameters
# rand_search = RandomizedSearchCV(rf, 
#                                  param_distributions = param_dist, 
#                                  n_iter=5, 
#                                  cv=5)

# # Fit the random search object to the data
# rand_search.fit(X_train, y_train)
# # Create a variable for the best model
# best_rf = rand_search.best_estimator_

# # Print the best hyperparameters
# print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'max_depth': 14, 'n_estimators': 350}
